In [ ]:
!pip install kafka-python
!pip install requests
!pip install pandas


In [ ]:

import pandas as pd
import requests
import json
import time
from time import sleep
from json import dumps
from kafka import KafkaProducer
import requests


In [ ]:
producer = KafkaProducer(bootstrap_servers=['13.233.157.88:9092'],
                         value_serializer=lambda x:
                         dumps(x).encode('utf-8'))


In [ ]:
API_KEY = '72c2c635cd9742258ff7ffef4b2072d7'
symbol = "AAPL"

In [ ]:
duration_seconds = 200
start_time = time.time()

In [ ]:
while True:
    current_time = time.time()
    if (current_time - start_time) > duration_seconds:
        print(f"\nReached {duration_seconds} seconds, stopping data fetch.")
        break

    try:
        url = f'https://api.twelvedata.com/price?symbol={symbol}&apikey={API_KEY}'
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            if "price" in data:
                message = {
                    "stock": symbol,
                    "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
                    "data": {
                        "price": data["price"]
                    }
                }

                # Send to Kafka
                producer.send('demo_test', value=message)
                producer.flush()
                print(f"Sent data for {symbol}: {message['data']}")
            else:
                print("Unexpected response format:", data)
        else:
            print("Failed to fetch data. HTTP Code:", response.status_code)

    except Exception as e:
        print("Error during fetch/send:", e)

    #
    time.sleep(15)

print("Done sending real-time data.")